In [1]:
from KeRLas import Brain, TimedGymEnv, Trainer
from KeRLas.models import defaultQModel
from KeRLas.models import DirectDiffModel, NaiveQModel, LateralDiffModel, DQN
from KeRLas.policies import BoltzmannQPolicy
import numpy as np
import gym, math
from simple_lander import SimpleLander

#env = GymEnv(CartPoleEnv(), tlimit=300)
env = TimedGymEnv(SimpleLander(), tlimit=500)

observation_width = env.observation_space.shape[0]
nactions = env.action_space.n


#for i in range(10):
#    next(brain.Memory.generate_samples(20))
#_ = next(brain.trainig_data_generator(mbsize))



Using TensorFlow backend.


WARN: gym.spaces.Box autodetected dtype as <type 'numpy.float32'>. Please provide explicit dtype.


In [6]:
rlmodel = LateralDiffModel(defaultQModel(observation_width, nactions), 0.99, 0.5)
policy = BoltzmannQPolicy(0.0001)
brain = Brain(rlmodel, policy, training_policies=map(BoltzmannQPolicy, [100.0, 10.0, 1.0, 0.01, 0.001]))


In [7]:
class Callback:
    
    def onEpisodeBegin(self, env, agents, observations):
        env.render()

    def onStep(self, env, env_done, feedback):
        env.render()
        
    def onEpisodeEnd(self, env, record, info):
        env.render()
        #print len(record), info
        
        #for (s0,a,s1,r,f), qv in zip(record, info["qvectors"]):
        #    print s0,a,s1,r,f, qv
        

trainer = Trainer(env, brain, 0.00, 10000)   

from KeRLas import GymPlayer
p = GymPlayer(env, brain, callback=Callback())




In [ ]:
mbsize = 50
for i in xrange(500):
    metrics = trainer.train(mbsize, 1000, 10)
    
    nsum = 0
    N = 5
    reward_sum = 0.0
    with brain.training(False):
        for _ in range(N):
            history = p.runEpisode()
            reward = sum([r for _,_,_,r,_ in history])
            #print reward, history[-1][3]
            reward_sum += reward
            #print len(history)
            nsum += len(history)
        print float(nsum)/N, reward_sum/N, metrics